# Exporting model from PyTorch to ONNX
从pytorch导出模型到onnx

In this tutorial, we describe how to use ONNX to convert a model defined
in PyTorch into the ONNX format.
在这个教程里面，我们将介绍如何使用ONNX将pytorch定义的模型转换为ONNX的模型
ONNX exporter is part of the [PyTorch repository](http://pytorch.org/docs/master/onnx.html).
ONNX导出器是PyTorch的一个模块

For working with this tutorial, you will need to install [onnx](https://github.com/onnx/onnx). You can get binary builds of onnx with
为了使用本教程，你需要安装:

``conda install -c conda-forge onnx``.

``NOTE``: ONNX is under active development so for the best support consider building PyTorch master branch which can be installed by following
[the instructions here](https://github.com/pytorch/pytorch#from-source)

## Invoking exporter
调用导出器

Pretty much it's a matter of replacing `my_model(input)` with `torch.onnx.export(my_model, input, "my_model.onnx")` in your script.
在你的脚本中，用torch.onnx.export（my_model，input，“my_model.onnx”）来代替my_model（input）
### Limitations
The ONNX exporter is a trace-based exporter,
ONNX导出器是一个基础跟踪导出器，

which means that it operates by executing your model once, and exporting the operators which were actually run during this run.
意思是当执行了一次你的模型，并且导出这次运行中的实际操作者。
This means that if your model is dynamic, e.g., changes behavior depending on input data, the export won’t be accurate.
这个意思是当你的模型是动态的，例如：根据输入数据改变行为，这个导出将是不准确的
Similarly, a trace is might be valid only for a specific input size (which is one reason why we require explicit inputs on tracing).
同样的，跟踪仅对特定的输入大小有效（这就是为什么我们要明确跟踪输入的一个原因）
Most of the operators export size-agnostic versions and should work on different batch sizes or input sizes. 
大部分的计算导出位置大小的版本，并且需要在不同的batch size和input size上工作
We recommend examining the model trace and making sure the traced operators look reasonable.
我们建议检查滨跟踪这个模型，确保跟踪的操作者看起来合理

In [1]:
import torch.onnx
help(torch.onnx.export)

Help on function export in module torch.onnx:

export(model, args, f, export_params=True, verbose=False, training=False)
    Export a model into ONNX format.  This exporter runs your model
    once in order to get a trace of its execution to be exported; at the
    moment, it does not support dynamic models (e.g., RNNs.)
    
    See also: :ref:`onnx-export`
    
    Arguments:
        model (torch.nn.Module): the model to be exported.
        args (tuple of arguments): the inputs to
            the model, e.g., such that ``model(*args)`` is a valid
            invocation of the model.  Any non-Variable arguments will
            be hard-coded into the exported model; any Variable arguments
            will become inputs of the exported model, in the order they
            occur in args.  If args is a Variable, this is equivalent
            to having called it with a 1-ary tuple of that Variable.
            (Note: passing keyword arguments to the model is not currently
            su

## Trying it out on AlexNet

If you already have your model built, it's just a few lines:

In [2]:
from torch.autograd import Variable
import torch.onnx
import torchvision
import netron

dummy_input = Variable(torch.randn(1, 3, 224, 224))

model = torchvision.models.alexnet(pretrained=True)

torch.onnx.export(model, dummy_input, "output/alexnet.onnx")

**That's it!**

## Inspecting model

You can also use ONNX tooling to check the validity of the resulting model or inspect the details

In [3]:
from torch.autograd import Variable
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
import random
import torch.onnx
import torchvision
price_data = pd.read_csv(open('dataset.csv'))
all_data = price_data.iloc[1:,2:3].values
train_data = price_data.iloc[1:151,2:3].values
test_data = price_data.iloc[1:152,2:3].values
train_data2 = price_data.iloc[2:222,2:3].values
print(train_data2.size)
# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
#dummy_input = Variable(torch.randn(1, 3, 224, 224))
# Obtain your model, it can be also constructed in your script explicitly
model = torch.load('net2.pkl')
input2 = Variable(torch.from_numpy(train_data2)).type(torch.FloatTensor)
dummy_input = input2.view(-1,1,1)
# Invoke export
torch.onnx.export(model,dummy_input, "output/rnn.onnx")

220


AttributeError: Can't get attribute 'RNN' on <module '__main__'>

Notice that all parameters are listed as graph's inputs but they also have stored values initialized in `model.graph.initializers`.

In [ ]:
## What's next
!du -h output/alexnet.onnx

In [ ]:

Check [PyTorch documentation on onnx file](http://pytorch.org/docs/master/onnx.html)
Take a look at [other tutorials, including importing of ONNX models to other frameworks](https://github.com/onnx/tutorials/tree/master/tutorials)